In [1]:
from transformers import ElectraTokenizer
from electraDataset import LazyElectraDataset
from simpletransformers.config.model_args import LanguageModelingArgs
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
tokenizer = ElectraTokenizer.from_pretrained('./vocab', do_lower_case=False, tokenize_chinese_chars=False)

In [51]:
args = LanguageModelingArgs()
args.update_from_dict({'model_type': 'electra', 'cache_dir': './cache_dir/', 'max_seq_length': 128})
dataset = LazyElectraDataset(tokenizer, args, "./inputs/pretrain/test.txt", "dev", 128)

985it [00:00, 3237.14it/s]


In [52]:
sampler = SequentialSampler(dataset)
loader = DataLoader(dataset, sampler=sampler, batch_size=2)
for batch in loader:
    break


In [53]:
labels = batch[0]
for tokens in labels.tolist():
    print(tokenizer.convert_ids_to_tokens(tokens))

['[CLS]', 'আস', '##্ল', '##ফ', '##জ', '##দল', '##াস', '##দ', '##ফ', 'ভাল', 'এন্টার', '##্ক', '##টিকা', 'অক্টোবর', '১০', '##th', '১৯০', '##৪', 'Fac', '##e', '১০', 'পর্যন্ত', 'প্রাচীর', 'গণ্য', '[UNK]', 'Ab', '##hi', '##k', 'ভাল', 'ছেলে', '।', 'আমি', 'বললাম', '12', '##3', '##4', ',', 'তুমি', 'একটা', 'গাধা', '।', '(', 'A', ')', ':', 'সে', 'একটা', 'ভাল', 'ছেলে', '।', '[SEP]', 'যারা', 'আগে', 'কৈর', '##ালার', 'শক্ত', 'নেতৃত্ব', 'দেয়ার', 'অক্ষমতা', '##র', 'সমালোচনা', 'করতো', 'এহ', '##ন', 'শান্তি', 'প্রক্রিয়াকে', '##ও', 'সঠিকভাবে', 'শেষ', 'করার', 'চ্যালেঞ্জের', 'সম্মুখীন', 'হচ্ছেন', '।', '[UNK]', '[UNK]', 'र', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '।', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'त', '##र', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'क', '##्र', '##म', '##को', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '।', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'

In [48]:
batch_masked_indices = []

for tokens in labels.tolist():
    candidate_index_sets = []
    for i, token in enumerate(tokens):
        if token in [tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id]:
            continue
        if candidate_index_sets and tokenizer._convert_id_to_token(token).startswith('##'):
            candidate_index_sets[-1].append((i, tokenizer._convert_id_to_token(token)))
        else:
            candidate_index_sets.append([(i, tokenizer._convert_id_to_token(token))])

    # print(candidate_index_sets)
    probability_vector = torch.full((1, len(candidate_index_sets)), args.mlm_probability)
    masked_vector = torch.bernoulli(probability_vector).bool().tolist()
    
    selected_indices = []
    for i, choice in enumerate(masked_vector[0]):
        if choice:
            print(candidate_index_sets[i])
            selected_indices.extend(candidate_index_sets[i])
    print('Final', len(candidate_index_sets))
    print(selected_indices)
    ids = [a for a, b in selected_indices]

    batch_masked_indices.append([1 if i in ids else 0 for i, _ in enumerate(tokens)])
    print(batch_masked_indices, len(batch_masked_indices[0]))

batch_masked_indices = torch.tensor(batch_masked_indices, dtype=torch.bool)
labels[~batch_masked_indices] = -100  # We only compute loss on masked tokens

[(9, 'ভাল')]
[(10, 'এন্টার'), (11, '##্ক'), (12, '##টিকা')]
[(14, '১০'), (15, '##th')]
[(33, 'বললাম')]
[(37, ',')]
[(44, ')')]
Final 33
[(9, 'ভাল'), (10, 'এন্টার'), (11, '##্ক'), (12, '##টিকা'), (14, '১০'), (15, '##th'), (33, 'বললাম'), (37, ','), (44, ')')]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] 64
[(11, 'করতো')]
[(12, 'এহ'), (13, '##ন')]
[(18, 'শেষ')]
[(32, '[UNK]')]
[(38, '[UNK]')]
[(47, 'त'), (48, '##र')]
[(53, '[UNK]')]
[(54, 'क'), (55, '##्र'), (56, '##म'), (57, '##को')]
[(59, '[UNK]')]
Final 54
[(11, 'করতো'), (12, 'এহ'), (13, '##ন'), (18, 'শেষ'), (32, '[UNK]'), (38, '[UNK]'), (47, 'त'), (48, '##र'), (53, '[UNK]'), (54, 'क'), (55, '##्र'), (56, '##म'), (57, '##को'), (59, '[UNK]')]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

In [49]:
labels

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  2230,
         29367,  1100, 31289,  -100,  2321,  2859,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100, 11671,  -100,  -100,  -100,    16,  -100,  -100,
          -100,  -100,  -100,  -100,    13,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100],
        [ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  7429, 22698,   516,  -100,  -100,  -100,  -100,  1706,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,     1,  -100,  -100,  -100,  -100,  -100,     1,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,   206,   819,  -100,
          -100,  -100,  -100,     1,   205, 12230,   822, 11660,  -100,     1,
          -100

In [50]:
for tokens in labels.tolist():
    print(tokenizer.convert_ids_to_tokens(tokens))

['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'ভাল', 'এন্টার', '##্ক', '##টিকা', '[UNK]', '১০', '##th', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'বললাম', '[UNK]', '[UNK]', '[UNK]', ',', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', ')', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]']
['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'করতো', 'এহ', '##ন', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'শেষ', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'त', '##र', '[UNK]',

In [35]:
tokenizer.sep_token_id, tokenizer.mask_token_id, tokenizer.cls_token_id, tokenizer.unk_token_id

(3, 4, 2, 1)

In [3]:
import json
data = json.loads(
    '''[[2, 29491, 19932, 6580, 6876, 116, 20296, 5304, 9239, 4762, 1900, 1913, 420, 2358, 12205, 525, 6876, 8962, 870, 29491, 19932, 5408, 1406, 1402, 4619, 2768, 406, 3051, 2423, 15647, 9254, 20260, 21002, 19501, 517, 594, 525, 6876, 19311, 1601, 418, 5408, 1671, 1573, 406, 3697, 116, 4752, 976, 2762, 1856, 13061, 991, 1483, 418, 22941, 5408, 2505, 1058, 1031, 3374, 1058, 406, 1186, 5408, 3138, 1414, 6876, 936, 888, 19844, 2161, 116, 1020, 1004, 4277, 4091, 406, 1186, 930, 3353, 1139, 10615, 3170, 898, 2107, 406, 6876, 1115, 116, 111, 2073, 2006, 992, 1049, 5408, 908, 2392, 1130, 1536, 2798, 899, 406, 3771, 1516, 1166, 23343, 878, 2178, 116, 1644, 1166, 23343, 878, 554, 2036, 406, 1049, 3771, 1018, 10615, 3170, 898, 2178, 406, 111, 4619, 2423, 554, 8585, 930, 6999, 1115, 116, 111, 3539, 18204, 116, 2700, 888, 1130, 31474, 514, 7773, 2217, 406, 1004, 5984, 1516, 1129, 2390, 940, 418, 1828, 3353, 1388, 10247, 406, 992, 4522, 22300, 406, 1018, 1146, 1058, 116, 992, 4825, 1111, 406, 1018, 418, 27185, 7440, 1184, 406, 1282, 2278, 7313, 406, 2310, 1695, 3827, 1037, 1037, 116, 12518, 12518, 1138, 1567, 1232, 2157, 1055, 406, 111, 3899, 6876, 8962, 870, 22941, 418, 5408, 2505, 547, 17273, 1591, 9239, 3763, 6589, 406, 3763, 7249, 2131, 2208, 13490, 4479, 4409, 1589, 898, 2161, 116, 111, 1235, 1241, 8032, 4619, 2423, 6574, 1402, 116, 3709, 2392, 1130, 5108, 30801, 950, 406, 6876, 870, 5408, 1010, 1122, 6574, 936, 20323, 862, 11069, 6707, 105, 111, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
'''
)
data = data[0]


In [5]:
tokenizer.decode(data)

"[CLS] সম্মানসূচক ডিলিট পেলেন মমতা, বিরোধীদের সমালোচনা পশ্চিমবঙ্গের মূখ্যমন্ত্রী ও শিক্ষা অনুরাগী মমতা বন্দ্যোপাধ্যায়কে সম্মানসূচক ডিলিট ডিগ্রি প্রধান করেছে কলকাতা বিশ্ববিদ্যালয় । বৃহস্পতিবার বিশ্ববিদ্যালয়ের আচার্য কেশরীনাথ ত্রিপাঠী মমতা বন্দ্যোপাধ্যায়ের হাতে এ ডিগ্রি তুলে দেন । সাহিত্য, সংস্কৃতি এবং সামাজিক ক্ষেত্রে অবদানের জন্য তাকে এ সম্মানজনক ডিগ্রি দেয়া হয়েছে বলে জানানো হয়েছে । তবে ডিগ্রি পাওয়ার পরে মমতা তার এক প্রতিক্রিয়ায় বলেছেন, তিনি এর যোগ্য নন । তবে এই সম্মান তাঁর জীবনকে পূর্ণ করে দিয়েছে । মমতা বলেন,'বিশ্বাস করুন আমি কিন্তু ডিগ্রিটা কোনও দিন ব্যবহার করব না । আপনারা আমাকে সাম্মানিক দিয়েছেন, সেটা সাম্মানিকই থাকবে । কিন্তু আপনারা আমার জীবনকে পূর্ণ করে দিয়েছেন ।'কলকাতা বিশ্ববিদ্যালয়েরই প্রাক্তন এই ছাত্রী বলেন,'কখনও ভাবিনি, এভাবে এক দিন সমাবর্তনে ভাষণ দেব । এর আগেও আমাকে অনেক প্রতিষ্ঠান থেকে এ ধরনের সম্মান দিতে চেয়েছিল । আমি রাজি হইনি । আমার মনে হয়েছে, আমি ক্ষুদ্র মানুষ । আমার এ সবের যোগ্যতা নেই । তাই গ্রহণ করিনি । সারা জীবন লড়াই করতে করতে, মরতে মরতে আজ এখানে এসে দাঁড়িয়েছি ।'অন্যদ

In [9]:
a = '''
সম্মানসূচক ডিলিট পেলেন মমতা, বিরোধীদের সমালোচনা পশ্চিমবঙ্গের মূখ্যমন্ত্রী ও শিক্ষা অনুরাগী মমতা বন্দ্যোপাধ্যায়কে সম্মানসূচক ডিলিট ডিগ্রি প্রধান করেছে কলকাতা বিশ্ববিদ্যালয় । বৃহস্পতিবার বিশ্ববিদ্যালয়ের আচার্য কেশরীনাথ ত্রিপাঠী মমতা বন্দ্যোপাধ্যায়ের হাতে এ ডিগ্রি তুলে দেন । সাহিত্য, সংস্কৃতি এবং সামাজিক ক্ষেত্রে অবদানের জন্য তাকে এ সম্মানজনক ডিগ্রি দেয়া হয়েছে বলে জানানো হয়েছে । তবে ডিগ্রি পাওয়ার পরে মমতা তার এক প্রতিক্রিয়ায় বলেছেন, তিনি এর যোগ্য নন । তবে এই সম্মান তাঁর জীবনকে পূর্ণ করে দিয়েছে । মমতা বলেন,'বিশ্বাস করুন আমি কিন্তু ডিগ্রিটা কোনও দিন ব্যবহার করব না । আপনারা আমাকে সাম্মানিক দিয়েছেন, সেটা সাম্মানিকই থাকবে । কিন্তু আপনারা আমার জীবনকে পূর্ণ করে দিয়েছেন ।'কলকাতা বিশ্ববিদ্যালয়েরই প্রাক্তন এই ছাত্রী বলেন,'কখনও ভাবিনি, এভাবে এক দিন সমাবর্তনে ভাষণ দেব । এর আগেও আমাকে অনেক প্রতিষ্ঠান থেকে এ ধরনের সম্মান দিতে চেয়েছিল । আমি রাজি হইনি । আমার মনে হয়েছে, আমি ক্ষুদ্র মানুষ । আমার এ সবের যোগ্যতা নেই । তাই গ্রহণ করিনি । সারা জীবন লড়াই করতে করতে, মরতে মরতে আজ এখানে এসে দাঁড়িয়েছি ।'অন্যদিকে মমতা বন্দ্যোপাধ্যায়কে সম্মানজনক এ ডিগ্রি দেয়ায় কটাক্ষ করছে পশ্চিমবঙ্গের রাজ্য বিজেপি । রাজ্য বিজিপির সভাপতি দিলীপ ঘোষ বিদ্রুপ করে বলেছেন,'সরকার এমন একজনকে কলকাতা বিশ্ববিদ্যালয়ের উপাচার্য করেছে, যিনি কোনও দিন কলেজে পড়াননি । মমতাকে ডিগ্রি দিয়ে সেই উপাচার্য তার প্রতিদানের কৃতজ্ঞতা রেখেছেন!'
'''


In [10]:
len(a)

1287